In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

In [ ]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_10c import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')